In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, Layout, HBox, VBox
from IPython.display import display, HTML

# --- 初期パラメータ設定 ---
initial_w = 1.0
initial_k1 = 1.0
initial_k2 = 0.5

# 空間範囲
x_min, x_max = -5, 5
y_min, y_max = -5, 5
num_points = 80 # 解像度

# 時間範囲 (スライダー用)
t_min, t_max = 0, 2 * np.pi / initial_w # 少なくとも1周期分
t_step = 0.1

# --- 空間グリッドの生成 ---
x = np.linspace(x_min, x_max, num_points)
y = np.linspace(y_min, y_max, num_points)
X, Y = np.meshgrid(x, y)

# --- Plotly Figureの初期化 ---
# 初期波形を計算
initial_u = np.cos(initial_w * 0 - initial_k1 * X - initial_k2 * Y)

fig = go.Figure(data=[go.Surface(z=initial_u, x=X, y=Y, colorscale='Viridis')])

# レイアウトの設定
fig.update_layout(
    title=r'$$u(x, y, t) = \cos(\omega t - k_1 x - k_2 y)$$',
    scene=dict(
        xaxis_title=r'$$x$$',
        yaxis_title=r'$$y$$',
        zaxis_title=r'$$u$$',
        zaxis=dict(range=[-1.2, 1.2]), # Z軸の範囲を固定
        aspectmode='auto' # 3Dのアスペクト比自動調整
    ),
    width=800,
    height=700,
    margin=dict(l=50, r=50, b=50, t=80),
    # Annotations for 2D layout should be placed here, not in scene
    annotations=[ # 初期アノテーションを設定
        dict(
            text=f'Time: $t={0:.2f}$', # 初期時刻
            xref="paper", yref="paper",
            x=0.05, y=0.95, showarrow=False,
            font=dict(size=14, color="black")
        )
    ]
)

# --- インタラクティブUIの作成 ---

# スライダーのスタイル設定
slider_layout = Layout(width='600px')

# 角周波数 w のスライダー
w_slider = FloatSlider(
    value=initial_w,
    min=0.1,
    max=5.0,
    step=0.1,
    description='Angular Frequency (w):',
    orientation='horizontal',
    layout=slider_layout
)

# 波数 k1 のスライダー
k1_slider = FloatSlider(
    value=initial_k1,
    min=0.0,
    max=3.0,
    step=0.1,
    description='Wave Number k1:',
    orientation='horizontal',
    layout=slider_layout
)

# 波数 k2 のスライダー
k2_slider = FloatSlider(
    value=initial_k2,
    min=0.0,
    max=3.0,
    step=0.1,
    description='Wave Number k2:',
    orientation='horizontal',
    layout=slider_layout
)

# 時間 t のスライダー
t_slider = FloatSlider(
    value=0,
    min=t_min,
    max=t_max,
    step=t_step,
    description='Time (t):',
    orientation='horizontal',
    layout=slider_layout
)

# --- 更新関数 ---
def update_plot(t_val, w_val, k1_val, k2_val):
    """
    スライダーの値に基づいてプロットを更新する関数
    """
    # 新しいZデータを計算
    new_u = np.cos(w_val * t_val - k1_val * X - k2_val * Y)

    # Plotlyのトレースを更新
    with fig.batch_update(): # バッチアップデートで高速化
        fig.data[0].z = new_u
        fig.layout.title.text = (
            f'$$u(x, y, t) = \\cos({w_val:.1f}t - {k1_val:.1f}x - {k2_val:.1f}y)$$'
        )
        # 時間表示のアノテーションを更新 (fig.layout.annotations[0] を直接更新)
        fig.layout.annotations[0].text = f'Time: $t={t_val:.2f}$'
        # ここでは他のプロパティ（xref, yrefなど）は初期設定のままで変更しない
        # fig.layout.annotations[0].xref = "paper" # これらは初期設定済みなので不要
        # fig.layout.annotations[0].yref = "paper" # これらは初期設定済みなので不要


# --- UIとプロットの結合 ---
# interact関数を使ってスライダーとupdate_plot関数を結合
interactive_plot = interact(
    update_plot,
    t_val=t_slider,
    w_val=w_slider,
    k1_val=k1_slider,
    k2_val=k2_slider
)

# UIウィジェットとFigureを縦に並べて表示
ui = VBox([
    HBox([w_slider]),
    HBox([k1_slider]),
    HBox([k2_slider]),
    HBox([t_slider])
])

# Jupyter Notebook/Labで表示
display(ui, fig)

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Animated GDP and population over decades'),
    html.P("Select an animation:"),
    dcc.RadioItems(
        id='selection',
        options=["GDP - Scatter", "Population - Bar"],
        value='GDP - Scatter',
    ),
    dcc.Loading(dcc.Graph(id="graph"), type="cube")
])


@app.callback(
    Output("graph", "figure"),
    Input("selection", "value"))
def display_animated_graph(selection):
    df = px.data.gapminder() # replace with your own data source
    animations = {
        'GDP - Scatter': px.scatter(
            df, x="gdpPercap", y="lifeExp", animation_frame="year",
            animation_group="country", size="pop", color="continent",
            hover_name="country", log_x=True, size_max=55,
            range_x=[100,100000], range_y=[25,90]),
        'Population - Bar': px.bar(
            df, x="continent", y="pop", color="continent",
            animation_frame="year", animation_group="country",
            range_y=[0,4000000000]),
    }
    return animations[selection]


app.run(debug=True)

In [ ]:
import plotly.graph_objects as go

import numpy as np


# Generate curve data
t = np.linspace(-1, 1, 100)
x = t + t ** 2
y = t - t ** 2
xm = np.min(x) - 1.5
xM = np.max(x) + 1.5
ym = np.min(y) - 1.5
yM = np.max(y) + 1.5
N = 50
s = np.linspace(-1, 1, N)
xx = s + s ** 2
yy = s - s ** 2
vx = 1 + 2 * s
vy = 1 - 2 * s  # v=(vx, vy) is the velocity
speed = np.sqrt(vx ** 2 + vy ** 2)
ux = vx / speed  # (ux, uy) unit tangent vector, (-uy, ux) unit normal vector
uy = vy / speed

xend = xx + ux  # end coordinates for the unit tangent vector at (xx, yy)
yend = yy + uy

xnoe = xx - uy  # end coordinates for the unit normal vector at (xx,yy)
ynoe = yy + ux


# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     name="frame",
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=x, y=y,
                     name="curve",
                     mode="lines",
                     line=dict(width=2, color="blue"))
          ],
    layout=go.Layout(width=600, height=600,
                     xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
                     yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
                     title=dict(text="Moving Frenet Frame Along a Planar Curve"),
                     hovermode="closest",
                     updatemenus=[dict(type="buttons",
                                       buttons=[dict(label="Play",
                                                     method="animate",
                                                     args=[None])])]),

    frames=[go.Frame(
        data=[go.Scatter(
            x=[xx[k], xend[k], None, xx[k], xnoe[k]],
            y=[yy[k], yend[k], None, yy[k], ynoe[k]],
            mode="lines",
            line=dict(color="red", width=2))
        ]) for k in range(N)]
)

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[go.Scatter(x=[0, 1], y=[0, 1])],
    layout=go.Layout(
        xaxis=dict(range=[0, 5], autorange=False),
        yaxis=dict(range=[0, 5], autorange=False),
        title=dict(text="Start Title"),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=[1, 2], y=[1, 2])]),
            go.Frame(data=[go.Scatter(x=[1, 4], y=[1, 4])]),
            go.Frame(data=[go.Scatter(x=[3, 4], y=[3, 4])],
                     layout=go.Layout(title_text="End Title"))]
)

fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np

# パラメータ設定
omega = 1
k1 = 0.5
k2 = 2

# 空間範囲の設定
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)

# 時間範囲の設定
time_steps = np.linspace(0, 2 * np.pi, 50) # 0から2πまで50ステップ

# 初期フレームのデータ生成
initial_t = time_steps[0]
Z_initial = np.cos(omega * initial_t - k1 * X - k2 * Y)

# フレームのリストを作成
frames = []
for t in time_steps:
    Z = np.cos(omega * t - k1 * X - k2 * Y)
    frames.append(go.Frame(data=[go.Surface(z=Z, x=X, y=Y,
                                             colorscale='viridis',  # 色スケール
                                             cmin=-1, cmax=1)], # z値の範囲を固定
                           name=str(t))) # フレーム名（任意）

# 初期プロットの作成
fig = go.Figure(
    data=[go.Surface(z=Z_initial, x=X, y=Y,
                     colorscale='viridis',
                     cmin=-1, cmax=1)],
    layout=go.Layout(
        title='3D Plane Wave Animation',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='u',
            zaxis=dict(range=[-1.5, 1.5]) # z軸の範囲を固定してアニメーション中のスケール変更を防ぐ
        ),
        width=900,
        height=700,
        updatemenus=[dict(
            type='buttons',
            buttons=[dict(label='Play',
                          method='animate',
                          args=[None, {"frame": {"duration": 50, "redraw": True},
                                       "fromcurrent": True, "transition": {"duration": 0, "easing": "linear"}}])])]
    ),
    frames=frames
)

fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np

# 固定の空間範囲
x_range = np.linspace(-5, 5, 100)
y_range = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x_range, y_range)

# 初期パラメータ
initial_omega = 1
initial_k1 = 0.5
initial_k2 = 2

# 時間ステップ (アニメーション用)
time_steps = np.linspace(0, 2 * np.pi, 50)

def generate_surface_data(omega, k1, k2, t):
    """指定されたパラメータと時間で平面波のZ値を計算する関数"""
    Z = np.cos(omega * t - k1 * X - k2 * Y)
    return Z

# 初期状態のZ値
initial_Z = generate_surface_data(initial_omega, initial_k1, initial_k2, time_steps[0])

# フレームのリストを作成 (アニメーション用)
frames = []
for t_anim in time_steps:
    Z_anim = generate_surface_data(initial_omega, initial_k1, initial_k2, t_anim)
    frames.append(go.Frame(data=[go.Surface(z=Z_anim, x=X, y=Y,
                                             colorscale='viridis',
                                             cmin=-1, cmax=1)],
                           name=f"time_{t_anim:.2f}"))

# スライダーのステップを生成するヘルパー関数
def create_slider_steps(param_name, current_val, min_val, max_val, num_steps, is_int=False):
    steps = []
    values = np.linspace(min_val, max_val, num_steps)
    if is_int:
        values = np.round(values).astype(int)

    for val in values:
        # スライダーを動かしたときに、現在の時間ステップと他のパラメータを維持しつつ、
        # 対象のパラメータだけを更新するように設定
        # ただし、アニメーション中は時間が変化するため、スライダーの更新は静止画として機能します
        # 複雑な連動はDashが必要です。
        steps.append(
            dict(
                method='restyle',
                label=f"{val:.2f}" if not is_int else str(val),
                args=[{'z': [generate_surface_data(
                    initial_omega if param_name != 'omega' else val,
                    initial_k1 if param_name != 'k1' else val,
                    initial_k2 if param_name != 'k2' else val,
                    time_steps[0] # スライダーでの更新は初期時間での静止画として扱う
                )]}, [0]] # 0番目のトレース (Surface) を更新
            )
        )
    return steps


# スライダーの定義
sliders_list = [
    dict(
        active=initial_omega,
        currentvalue={"prefix": "Omega: "},
        pad={"t": 80},
        steps=create_slider_steps('omega', initial_omega, 0.1, 5.0, 50)
    ),
    dict(
        active=initial_k1,
        currentvalue={"prefix": "k1: "},
        pad={"t": 160}, # スライダー間のスペースを調整
        steps=create_slider_steps('k1', initial_k1, 0.1, 5.0, 50)
    ),
    dict(
        active=initial_k2,
        currentvalue={"prefix": "k2: "},
        pad={"t": 240}, # さらにスペースを調整
        steps=create_slider_steps('k2', initial_k2, 0.1, 5.0, 50)
    )
]


fig = go.Figure(
    data=[go.Surface(z=initial_Z, x=X, y=Y,
                     colorscale='viridis',
                     cmin=-1, cmax=1)],
    layout=go.Layout(
        title='3D Plane Wave Animation with Parameter Sliders',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='u',
            zaxis=dict(range=[-1.5, 1.5])
        ),
        width=900,
        height=700,
        updatemenus=[dict(
            type='buttons',
            buttons=[dict(label='Play Animation',
                          method='animate',
                          args=[None, {"frame": {"duration": 50, "redraw": True},
                                       "fromcurrent": True, "transition": {"duration": 0, "easing": "linear"}}])],
            x=0.05, # ボタンの位置調整
            y=1.05 # ボタンの位置調整
        )],
        sliders=sliders_list # スライダーを追加
    ),
    frames=frames
)

fig.show()

In [26]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objects as go
import numpy as np

# Dashアプリケーションの初期化
app = dash.Dash(__name__)

# --- 定数と初期値 ---
X_RANGE = np.linspace(-5, 5, 100)
Y_RANGE = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(X_RANGE, Y_RANGE)
TIME_STEPS = np.linspace(0, 2 * np.pi, 50) # アニメーションの時間ステップ

INITIAL_OMEGA = 1.0
INITIAL_K1 = 0.5
INITIAL_K2 = 2.0

# --- 平面波の計算関数 ---
def generate_surface_data(omega, k1, k2, t):
    """指定されたパラメータと時間で平面波のZ値を計算する関数"""
    Z = np.cos(omega * t - k1 * X - k2 * Y)
    return Z

# --- レイアウトの定義 ---
app.layout = html.Div([
    html.H1("3D Plane Wave Animator with Dash", style={'textAlign': 'center'}),

    # スライダーのコンテナ
    html.Div([
        html.Label("Omega (ω):"),
        dcc.Slider(
            id='omega-slider',
            min=0.1,
            max=5.0,
            step=0.1,
            value=INITIAL_OMEGA,
            marks={i: f'{i:.1f}' for i in np.arange(0.1, 5.1, 0.5)},
            tooltip={"placement": "bottom", "always_visible": True},
            updatemode='drag'
        ),
        html.Br(), # スペース

        html.Label("k1:"),
        dcc.Slider(
            id='k1-slider',
            min=-5.0, # 変更点: 最小値を-5.0に
            max=5.0,  # 変更点: 最大値を5.0に
            step=0.1,
            value=INITIAL_K1,
            marks={i: f'{i:.1f}' for i in np.arange(-5.0, 5.1, 1.0)}, # 変更点: 目盛りも調整
            tooltip={"placement": "bottom", "always_visible": True},
            updatemode='drag'
        ),
        html.Br(),

        html.Label("k2:"),
        dcc.Slider(
            id='k2-slider',
            min=-5.0, # 変更点: 最小値を-5.0に
            max=5.0,  # 変更点: 最大値を5.0に
            step=0.1,
            value=INITIAL_K2,
            marks={i: f'{i:.1f}' for i in np.arange(-5.0, 5.1, 1.0)}, # 変更点: 目盛りも調整
            tooltip={"placement": "bottom", "always_visible": True},
            updatemode='drag'
        ),
    ], style={'width': '80%', 'margin': 'auto', 'padding': '20px'}),

    # グラフ表示領域
    dcc.Graph(id='plane-wave-graph', style={'height': '600px'}),

    # アニメーション制御用のDCC Intervalとボタン
    html.Div([
        html.Button('Play Animation', id='play-button', n_clicks=0,
                    style={'marginRight': '10px'}),
        html.Button('Stop Animation', id='stop-button', n_clicks=0),
        dcc.Interval(
            id='interval-component',
            interval=50,  # ミリ秒 (20 FPS)
            n_intervals=0,
            disabled=True # 最初は無効化
        ),
        html.Div(id='current-time-display', style={'marginTop': '10px'})
    ], style={'textAlign': 'center', 'padding': '20px'})
])

# --- コールバックの定義 ---

# グラフ更新コールバック（スライダーまたは時間経過による更新）
@app.callback(
    Output('plane-wave-graph', 'figure'),
    Output('current-time-display', 'children'),
    Input('omega-slider', 'value'),
    Input('k1-slider', 'value'),
    Input('k2-slider', 'value'),
    Input('interval-component', 'n_intervals')
)
def update_graph(omega, k1, k2, n_intervals):
    t_index = n_intervals % len(TIME_STEPS)
    current_t = TIME_STEPS[t_index]

    Z = generate_surface_data(omega, k1, k2, current_t)

    fig = go.Figure(
        data=[go.Surface(z=Z, x=X, y=Y,
                         colorscale='viridis',
                         cmin=-1, cmax=1)],
        layout=go.Layout(
            title=f'Plane Wave: ω={omega:.2f}, k1={k1:.2f}, k2={k2:.2f}, t={current_t:.2f}',
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='u',
                zaxis=dict(range=[-2.0, 2.0])
            ),
            margin=dict(l=0, r=0, b=0, t=40)
        )
    )
    current_time_text = f"Current Animation Time: {current_t:.2f} s"
    return fig, current_time_text

# アニメーション制御コールバック
@app.callback(
    Output('interval-component', 'disabled'),
    Input('play-button', 'n_clicks'),
    Input('stop-button', 'n_clicks'),
    State('interval-component', 'disabled')
)
def toggle_animation(play_clicks, stop_clicks, is_disabled):
    ctx = dash.callback_context

    if not ctx.triggered:
        return True
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'play-button':
        return False
    elif button_id == 'stop-button':
        return True
    
    return is_disabled

# アプリケーションの実行
if __name__ == '__main__':
    app.run(debug=True) # 変更点: run_server() から run() に

In [37]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objects as go
import numpy as np

# Dashアプリケーションの初期化
app = dash.Dash(__name__)

# --- 定数と初期値 ---
X_RANGE = np.linspace(-5, 5, 100)
Y_RANGE = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(X_RANGE, Y_RANGE)
TIME_STEPS = np.linspace(0, 2 * np.pi, 50) # アニメーションの時間ステップ

INITIAL_OMEGA = 1.0
INITIAL_K1 = 0.5
INITIAL_K2 = 2.0

# --- 平面波の計算関数 ---
def generate_surface_data(omega, k1, k2, t):
    """指定されたパラメータと時間で平面波のZ値を計算する関数"""
    Z = np.cos(omega * t - k1 * X - k2 * Y)
    return Z

# --- レイアウトの定義 ---
app.layout = html.Div([
    html.H1("3D Plane Wave Animator with Dash", style={'textAlign': 'center'}),

    # スライダーのコンテナ
    html.Div([
        html.Label("Omega (ω):"),
        dcc.Slider(
            id='omega-slider',
            min=0.1,
            max=5.0,
            step=0.1,
            value=INITIAL_OMEGA,
            marks={i: f'{i:.1f}' for i in np.arange(0.1, 5.1, 0.5)},
            tooltip={"placement": "bottom", "always_visible": True},
            updatemode='drag'
        ),
        html.Br(),

        html.Label("k1:"),
        dcc.Slider(
            id='k1-slider',
            min=-5.0,
            max=5.0,
            step=0.1,
            value=INITIAL_K1,
            marks={i: f'{i:.1f}' for i in np.arange(-5.0, 5.1, 1.0)},
            tooltip={"placement": "bottom", "always_visible": True},
            updatemode='drag'
        ),
        html.Br(),

        html.Label("k2:"),
        dcc.Slider(
            id='k2-slider',
            min=-5.0,
            max=5.0,
            step=0.1,
            value=INITIAL_K2,
            marks={i: f'{i:.1f}' for i in np.arange(-5.0, 5.1, 1.0)},
            tooltip={"placement": "bottom", "always_visible": True},
            updatemode='drag'
        ),
    ], style={'width': '80%', 'margin': 'auto', 'padding': '20px'}),

    # グラフ表示領域
    dcc.Graph(id='plane-wave-graph', style={'height': '450px'}),

    # アニメーション制御用のDCC Intervalとボタン
    html.Div([
        html.Button('Play Animation', id='play-button', n_clicks=0,
                    style={'marginRight': '10px'}),
        html.Button('Stop Animation', id='stop-button', n_clicks=0),
        dcc.Interval(
            id='interval-component',
            interval=50,
            n_intervals=0,
            disabled=True
        ),
    ], style={'textAlign': 'center', 'padding': '20px'})
])

# --- コールバックの定義 ---

# グラフ更新コールバック（スライダーまたは時間経過による更新）
@app.callback(
    Output('plane-wave-graph', 'figure'),
    Input('omega-slider', 'value'),
    Input('k1-slider', 'value'),
    Input('k2-slider', 'value'),
    Input('interval-component', 'n_intervals')
)
def update_graph(omega, k1, k2, n_intervals):
    t_index = n_intervals % len(TIME_STEPS)
    current_t = TIME_STEPS[t_index]

    Z = generate_surface_data(omega, k1, k2, current_t)

    fig = go.Figure(
        data=[go.Surface(z=Z, x=X, y=Y,
                         colorscale='viridis',
                         cmin=-1, cmax=1)],
        layout=go.Layout(
            # 変更点: タイトルに時間表示を再追加
            title=f'Plane Wave: ω={omega:.2f}, k1={k1:.2f}, k2={k2:.2f}, t={current_t:.2f}',
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='u',
                zaxis=dict(range=[-1.5, 1.5])
            ),
            margin=dict(l=0, r=0, b=0, t=40)
        )
    )
    return fig

# アニメーション制御コールバック
@app.callback(
    Output('interval-component', 'disabled'),
    Input('play-button', 'n_clicks'),
    Input('stop-button', 'n_clicks'),
    State('interval-component', 'disabled')
)
def toggle_animation(play_clicks, stop_clicks, is_disabled):
    ctx = dash.callback_context

    if not ctx.triggered:
        return True
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'play-button':
        return False
    elif button_id == 'stop-button':
        return True
    
    return is_disabled

# アプリケーションの実行
if __name__ == '__main__':
    app.run(debug=True)